In [25]:
import pandas as pd
import numpy as np

In [27]:
# to do a simple test to check the dataset columns and corresponding data types
df = pd.read_csv('2010-2011 Solar home electricity data.csv', skiprows=1, header=0, low_memory=False)
print("\nColumns:")
print(df.columns.tolist())
df.dtypes


Columns:
['Customer', 'Generator Capacity', 'Postcode', 'Consumption Category', 'date', '0:30', '1:00', '1:30', '2:00', '2:30', '3:00', '3:30', '4:00', '4:30', '5:00', '5:30', '6:00', '6:30', '7:00', '7:30', '8:00', '8:30', '9:00', '9:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30', '14:00', '14:30', '15:00', '15:30', '16:00', '16:30', '17:00', '17:30', '18:00', '18:30', '19:00', '19:30', '20:00', '20:30', '21:00', '21:30', '22:00', '22:30', '23:00', '23:30', '0:00']


Customer                  int64
Generator Capacity      float64
Postcode                  int64
Consumption Category     object
date                     object
0:30                    float64
1:00                    float64
1:30                    float64
2:00                    float64
2:30                    float64
3:00                    float64
3:30                    float64
4:00                    float64
4:30                    float64
5:00                    float64
5:30                    float64
6:00                    float64
6:30                    float64
7:00                    float64
7:30                    float64
8:00                    float64
8:30                    float64
9:00                    float64
9:30                    float64
10:00                   float64
10:30                   float64
11:00                   float64
11:30                   float64
12:00                   float64
12:30                   float64
13:00                   float64
13:30   

In [28]:
df.columns

Index(['Customer', 'Generator Capacity', 'Postcode', 'Consumption Category',
       'date', '0:30', '1:00', '1:30', '2:00', '2:30', '3:00', '3:30', '4:00',
       '4:30', '5:00', '5:30', '6:00', '6:30', '7:00', '7:30', '8:00', '8:30',
       '9:00', '9:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30',
       '13:00', '13:30', '14:00', '14:30', '15:00', '15:30', '16:00', '16:30',
       '17:00', '17:30', '18:00', '18:30', '19:00', '19:30', '20:00', '20:30',
       '21:00', '21:30', '22:00', '22:30', '23:00', '23:30', '0:00'],
      dtype='object')

In [31]:
df.head()

,Customer,Generator Capacity,Postcode,Consumption Category,date,0:30,1:00,1:30,2:00,2:30,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00
0,1,3.78,2076,GC,1-Jul-10,0.303,0.471,0.083,0.121,0.361,...,0.495,0.54,0.406,0.543,0.495,0.216,0.378,0.128,0.078,0.125
1,1,3.78,2076,CL,1-Jul-10,1.250,1.244,1.256,0.744,0.019,...,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.075
2,1,3.78,2076,GG,1-Jul-10,0.000,0.000,0.000,0.000,0.000,...,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1,3.78,2076,GC,2-Jul-10,0.116,0.346,0.122,0.079,0.120,...,0.262,1.10,1.012,0.817,0.526,0.335,0.402,0.142,0.120,0.111
4,1,3.78,2076,CL,2-Jul-10,1.238,1.238,1.256,1.250,0.169,...,0.000,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.088


In [33]:
import pandas as pd
import numpy as np

In [35]:
def process_solar_file(file_path):
    """
    filter customer based on CL values and do date data format change
    """
    # read the csv file
    df = pd.read_csv(file_path, 
                        skiprows=1,
                        header=0,
                        low_memory=False)
        
    print(f"\nProcessing {file_path}")

    # get time columns
    time_cols = [col for col in df.columns if ':' in str(col)]
        
     
    # Find customers that have any CL > 0
    cl_mask = (df['Consumption Category'] == 'CL') & (df[time_cols] > 0).any(axis=1)
    customers_with_cl = df[cl_mask]['Customer'].unique()
    
    # Remove customers with CL > 0 and keep only GG/GC categories
    df = df[
        (~df['Customer'].isin(customers_with_cl)) & 
        (df['Consumption Category'].isin(['GG', 'GC']))
    ].copy()
    
    
    print(f"Number of customers removed due to positive CL values: {len(customers_with_cl)}")
        
    # convert date based on file year
    if '2010-2011' in file_path:
        df['date'] = pd.to_datetime(df['date'], format='%d-%b-%y')
    else:
        df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')


    # filter out rows where GG/GC values are all zero
    df = df[df[time_cols].sum(axis=1) > 0]
        
    return df

In [37]:
def create_daily_dataset(df):
    """
    create daily dataset with GG and GC values and corresponding date and net load
    """
    # get time columns
    time_cols = [col for col in df.columns if ':' in str(col)]
    
    # takes the first (and only) row of each group and then sums across all the time columns to get the daily total
    daily_sums = df.groupby(['Customer', 'Postcode', 'date', 'Consumption Category'])[time_cols].first().sum(axis=1).reset_index()
    
    # create the pivot table
    daily_data = daily_sums.pivot(
        index=['Customer', 'Postcode', 'date'],
        columns='Consumption Category',
        values=0
    ).reset_index()
    
    # In case any NaN values exist after the pivot operation, fill Nah with 0
    # and then we calculate the net load
    daily_data = daily_data.fillna(0)
    daily_data['net_load'] = daily_data['GC'] - daily_data['GG']

    daily_data.to_csv('prosumer_daily_data1.csv',
                     index=False,
                     sep=',',
                     date_format='%Y-%m-%d',
                     encoding='utf-8')
    
    return daily_data

In [39]:
def create_hourly_dataset(df):
    """
    create hourly dataset with GG and GC values and corresponding date and net load
    """
    # get time columns
    time_cols = [col for col in df.columns if ':' in str(col)]

    # Custom sorting function for time columns
    def time_to_minutes(time_str):
        # Handle the special case of '0:00' which should be last
        if time_str == '0:00':
            return 24 * 60
        hours, minutes = map(int, time_str.split(':'))
        return hours * 60 + minutes
    
    time_cols.sort(key=time_to_minutes)
    
    # melt to get hourly values
    melted = pd.melt(
        df,
        id_vars=['Customer', 'Postcode', 'date', 'Consumption Category'],
        value_vars=time_cols,  # convert from columns(half-hour time interval) to rows
        var_name='hour',       # new column name which contains the old column names
        value_name='value'     # new column name which contains the values
    )
    
    # create pivot table
    hourly_data = melted.pivot_table(
        index=['Customer', 'Postcode', 'date', 'hour'],
        columns='Consumption Category',
        values='value',
        fill_value=0
    ).reset_index()
    
    # add net load
    hourly_data['net_load'] = hourly_data['GC'] - hourly_data['GG']

    # Sort by Customer, date, and hour
    hourly_data = hourly_data.sort_values(['Customer', 'date', 'hour'], 
                                        key=lambda x: x if x.name != 'hour' 
                                        else pd.Series([time_to_minutes(t) for t in x]))

    hourly_data.to_csv('prosumer_hourly_data1.csv',
                      index=False,
                      sep=',',
                      date_format='%Y-%m-%d',
                      encoding='utf-8')
    
    return hourly_data

In [41]:
# dataset files to process
files = [
    '2010-2011 Solar home electricity data.csv',
    '2011-2012 Solar home electricity data v2.csv',
    '2012-2013 Solar home electricity data v2.csv'
]

# process all files
all_data = []
for file in files:
    df = process_solar_file(file)
    # check if we get valid data
    if df is not None and not df.empty:
        all_data.append(df)
        print(f"Successfully processed {file}")

if all_data:
    print("\nCombining all data...")
    # combine all years of data into one dataFrame
    combined_data = pd.concat(all_data, ignore_index=True)
    
    print("\nCreating daily dataset...")
    daily_data = create_daily_dataset(combined_data)
    
    print("\nCreating hourly dataset...")
    hourly_data = create_hourly_dataset(combined_data)
    
    print("\nFinal Summary:")
    print(f"Number of unique customers: {len(daily_data['Customer'].unique())}")
    print(f"Date range: {daily_data['date'].min()} to {daily_data['date'].max()}")
    print("\nDaily data columns:", daily_data.columns.tolist())
    print("Hourly data columns:", hourly_data.columns.tolist())
else:
    print("\nNo data was processed successfully.")


Processing 2010-2011 Solar home electricity data.csv
Number of customers removed due to positive CL values: 136
Successfully processed 2010-2011 Solar home electricity data.csv

Processing 2011-2012 Solar home electricity data v2.csv
Number of customers removed due to positive CL values: 136
Successfully processed 2011-2012 Solar home electricity data v2.csv

Processing 2012-2013 Solar home electricity data v2.csv
Number of customers removed due to positive CL values: 135
Successfully processed 2012-2013 Solar home electricity data v2.csv

Combining all data...

Creating daily dataset...

Creating hourly dataset...

Final Summary:
Number of unique customers: 166
Date range: 2010-07-01 00:00:00 to 2013-06-30 00:00:00

Daily data columns: ['Customer', 'Postcode', 'date', 'GC', 'GG', 'net_load']
Hourly data columns: ['Customer', 'Postcode', 'date', 'hour', 'GC', 'GG', 'net_load']


In [20]:
# different data columns like User ID, postcode, daily gg daily gc / User ID, postcode, hourly gg hourly gc
# data column formats required by fusen shixiong
# almost same as the before functions
def create_daily_dataset(df):
    """
    create daily dataset with format:
    User ID, postcode, daily GG, daily GC
    """
    # get time columns
    time_cols = [col for col in df.columns if ':' in str(col)]
    
    # calculate daily sums for GG and GC separately
    daily_gg = df[df['Consumption Category'] == 'GG'].groupby(['Customer', 'Postcode', 'date'])[time_cols].sum().sum(axis=1).reset_index()
    daily_gc = df[df['Consumption Category'] == 'GC'].groupby(['Customer', 'Postcode', 'date'])[time_cols].sum().sum(axis=1).reset_index()
    
    # merge GG and GC data
    daily_data = daily_gg.merge(daily_gc, on=['Customer', 'Postcode', 'date'], suffixes=('_gg', '_gc'))
    
    # rename columns to match desired format
    daily_data.columns = ['User ID', 'Postcode', 'date', 'daily_gg', 'daily_gc']
    
    # save with proper formatting
    daily_data.to_csv('prosumer_daily_data2.csv',
                     index=False,
                     sep=',',
                     #float_format='%.3f',
                     encoding='utf-8')
    
    return daily_data

def create_hourly_dataset(df):
    """
    create hourly dataset with format:
    User ID, postcode, hourly GG, hourly GC
    """
    # get time columns
    time_cols = [col for col in df.columns if ':' in str(col)]
    
    # melt the data to get hourly values
    melted = pd.melt(
        df,
        id_vars=['Customer', 'Postcode', 'date', 'Consumption Category'],
        value_vars=time_cols,
        var_name='hour',
        value_name='value'
    )
    
    # pivot to get GG and GC in separate columns
    hourly_data = melted.pivot_table(
        index=['Customer', 'Postcode', 'date', 'hour'],
        columns='Consumption Category',
        values='value',
        fill_value=0
    ).reset_index()
    
    # rename columns to match desired format
    hourly_data.columns.name = None
    hourly_data = hourly_data.rename(columns={
        'Customer': 'User ID',
        'Postcode': 'Postcode',
        'GG': 'hourly_gg',
        'GC': 'hourly_gc'
    })
    
    # sort by time
    def time_to_minutes(time_str):
        if time_str == '0:00':
            return 24 * 60
        hours, minutes = map(int, time_str.split(':'))
        return hours * 60 + minutes
    
    hourly_data = hourly_data.sort_values(
        ['User ID', 'date', 'hour'],
        key=lambda x: x if x.name != 'hour' else pd.Series([time_to_minutes(t) for t in x])
    )
    
    # save with proper formatting
    hourly_data.to_csv('prosumer_hourly_data2.csv',
                      index=False,
                      sep=',',
                      #float_format='%.3f',
                      encoding='utf-8')
    
    return hourly_data

# when processing the data:
if all_data:
    print("\nCombining all data...")
    combined_data = pd.concat(all_data, ignore_index=True)
    
    print("\nCreating daily dataset...")
    daily_data = create_daily_dataset(combined_data)
    
    print("\nCreating hourly dataset...")
    hourly_data = create_hourly_dataset(combined_data)
    
    print("\nFinal Summary:")
    print("\nDaily dataset format:")
    print(daily_data.head())
    print("\nHourly dataset format:")
    print(hourly_data.head())


Combining all data...

Creating daily dataset...

Creating hourly dataset...

Final Summary:

Daily dataset format:
   User ID  Postcode       date  daily_gg  daily_gc
0       11      2026 2010-07-01     6.743    26.873
1       11      2026 2010-07-02     1.977    20.961
2       11      2026 2010-07-03     7.305    37.023
3       11      2026 2010-07-04     6.573    31.955
4       11      2026 2010-07-05     1.378    34.751

Hourly dataset format:
    User ID  Postcode       date  hour  hourly_gc  hourly_gg
1        11      2026 2010-07-01  0:30      0.118        0.0
22       11      2026 2010-07-01  1:00      0.138        0.0
23       11      2026 2010-07-01  1:30      0.118        0.0
32       11      2026 2010-07-01  2:00      0.108        0.0
33       11      2026 2010-07-01  2:30      0.096        0.0
